<a href="https://colab.research.google.com/github/mohamedtal/PFE2021/blob/main/04_01_2021_CNN_DataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# before all don't forget to use the GPU

# at first we need to get the dataset from the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#unzip the dataset
# for the train
!unzip /content/drive/MyDrive/dataset/DR-dataset2015.zip


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: DR-dataset/train/2/3213_right.jpg  
  inflating: DR-dataset/train/2/15711_left.jpg  
  inflating: DR-dataset/train/2/43042_left.jpg  
  inflating: DR-dataset/train/2/13786_right.jpg  
  inflating: DR-dataset/train/2/40319_left.jpg  
  inflating: DR-dataset/train/2/13667_left.jpg  
  inflating: DR-dataset/train/2/3778_right.jpg  
  inflating: DR-dataset/train/2/44263_right.jpg  
  inflating: DR-dataset/train/2/10438_right.jpg  
  inflating: DR-dataset/train/2/25458_right.jpg  
  inflating: DR-dataset/train/2/40201_left.jpg  
  inflating: DR-dataset/train/2/32668_left.jpg  
  inflating: DR-dataset/train/2/19202_left.jpg  
  inflating: DR-dataset/train/2/22461_left.jpg  
  inflating: DR-dataset/train/2/40676_left.jpg  
  inflating: DR-dataset/train/2/41335_right.jpg  
  inflating: DR-dataset/train/2/10468_left.jpg  
  inflating: DR-dataset/train/2/2705_left.jpg  
  inflating: DR-dataset/train/2/3399

In [ ]:
# for the test
!unzip /content/drive/MyDrive/dataset/DR-2019-test.zip

In [8]:
!ls DR-dataset/train/2  | wc -l

5292


In [4]:
# we need to delete samples from class 0 ( there is 25000 images, we are going to take just 7000)
import random
import os
BASE_DIR = "DR-dataset/train/0"
NEW_DIR= "corbeille"

list_dir =  os.listdir(BASE_DIR)
to_val = random.sample(list_dir, len(list_dir) - 7000)
for images in to_val:
    os.rename(BASE_DIR + "/"+ images, NEW_DIR + "/" + images )

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:

# the train generator with data augmentation and preprocessing (rescale)
# https://xzz201920.medium.com/all-you-need-to-you-about-imagedatagenerator-in-keras-tensorflow-8fd436e4c0cd
# https://medium.com/swlh/data-augmentation-using-keras-4a852e49589f
# https://towardsdatascience.com/image-data-generators-in-keras-7c5fc6928400
# https://keras.io/api/preprocessing/image/
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

batchSize = 32

# ******** for train dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15)

train_generator = train_datagen.flow_from_directory(
    'DR-dataset/train/',
    classes = ['0','1','2','3','4'],
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=batchSize,
    class_mode='categorical',
    subset='training',
    shuffle=True)

# ******* for validation dataset
val_generator = train_datagen.flow_from_directory(
    'DR-dataset/train/',
    classes = ['0','1','2','3','4'],
    class_mode='categorical',
    color_mode='rgb',
    subset='validation'
)

# ******** for test dataset

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'DR-dataset2019-test/test/',
    classes = ['0','1','2','3','4'],
    target_size=(224, 224),
    class_mode='categorical',
    color_mode='rgb',
    batch_size = batchSize,
    shuffle = False)

Found 13871 images belonging to 5 classes.
Found 2445 images belonging to 5 classes.
Found 3662 images belonging to 5 classes.


In [11]:
num_classes = 5
img_size = (224,224,3)

# create the model
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras import Model
from tensorflow.python.keras.layers import Dense, Flatten

resnet = ResNet50(weights='imagenet', include_top=False, input_shape=img_size)

x = Flatten(input_shape=resnet.output.shape)(resnet.output)
x = Dense(1024, activation='sigmoid')(x)

predictions = Dense(num_classes, activation='softmax', name='pred')(x)
model = Model(inputs=[resnet.input], outputs=[predictions])

In [12]:
# compile the model and specify hyperparameter
# https://medium.com/ml-cheat-sheet/winning-at-loss-functions-common-loss-functions-that-you-should-know-a72c1802ecb4
# https://neptune.ai/blog/keras-loss-functions
lossFunction = 'categorical_crossentropy'
optimizeR = 'adam'
model.compile(loss = lossFunction, optimizer=optimizeR, metrics=['accuracy'])

In [13]:
# launch the train
import math
batchSize = 32
history = model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples//batchSize),
    epochs=50,
    validation_data = val_generator,
    validation_steps = 400,
    verbose = 1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
433/433 [==============================] - ETA: 0s - loss: 1.7283 - accuracy: 0.4065

InvalidArgumentError: ignored

In [ ]:
# saving the model
model.save("VGG_DR_06-01-2021.h5")

In [ ]:
# display the performance of the model graphically
import matplotlib.pyplot as plt
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()